In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

In [3]:
k=np.array([1,2,3,4])

In [4]:
print(k)

[1 2 3 4]


In [5]:
import pandas as pd

In [8]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [9]:
print("hello")

hello


In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
import string

# Load the email data
dataframe_emails = pd.read_csv('emails.csv')
dataframe_emails.head()

# Print basic statistics
print(f"Number of emails: {len(dataframe_emails)}")
print(f"Proportion of spam emails: {dataframe_emails.spam.sum()/len(dataframe_emails):.4f}")
print(f"Proportion of ham emails: {1 - dataframe_emails.spam.sum()/len(dataframe_emails):.4f}")

def preprocess_emails(df):
    """
    Preprocesses email data from a DataFrame.
    """
    df = df.sample(frac=1, ignore_index=True, random_state=42)
    X = df.text.apply(lambda x: x[9:]).to_numpy()  # Removing the "Subject:" prefix
    Y = df.spam.to_numpy()
    return X, Y

X, Y = preprocess_emails(dataframe_emails)

print(X[:5])
print(Y[:5])

email_index = 30
print(f"Email index {email_index}: {X[email_index]}\n\n")
print(f"Class: {Y[email_index]}")

def preprocess_text(X):
    """
    Preprocesses a collection of text data by removing stopwords and punctuation.
    """
    stop = set(stopwords.words('english') + list(string.punctuation))
    if isinstance(X, str):
        X = np.array([X])
    X_preprocessed = []
    for email in X:
        email = [i.lower() for i in word_tokenize(email) if i.lower() not in stop]
        X_preprocessed.append(email)
    return X_preprocessed

# This function may take a few seconds to run.
X_treated = preprocess_text(X)

email_index = 989
print(f"Email before preprocessing: {X[email_index]}")
print(f"Email after preprocessing: {X_treated[email_index]}")

# Split data into train and test sets
TRAIN_SIZE = int(0.80 * len(X_treated))  # 80% for training
X_train = X_treated[:TRAIN_SIZE]
Y_train = Y[:TRAIN_SIZE]
X_test = X_treated[TRAIN_SIZE:]
Y_test = Y[TRAIN_SIZE:]

print(f"Proportion of spam in train dataset: {sum(Y_train == 1) / len(Y_train):.4f}")
print(f"Proportion of spam in test dataset: {sum(Y_test == 1) / len(Y_test):.4f}")

def get_word_frequency(X, Y):
    """
    Calculate the frequency of each word in a set of emails categorized as spam (1) or not spam (0).
    """
    word_dict = {}
    num_emails = len(X)

    for i in range(num_emails):
        email = X[i]
        cls = Y[i]
        email_set = set(email)  # Avoid duplicates
        for word in email_set:
            if word not in word_dict:
                word_dict[word] = {'spam': 0, 'ham': 0}
            if cls == 0:  # ham
                word_dict[word]['ham'] += 1
            else:  # spam
                word_dict[word]['spam'] += 1

    return word_dict

word_frequency = get_word_frequency(X_train, Y_train)

# Class frequency
class_frequency = {'ham': sum(Y_train == 0), 'spam': sum(Y_train == 1)}

print(class_frequency)

# Proportion of spam in the training dataset
proportion_spam = class_frequency['spam'] / (class_frequency['ham'] + class_frequency['spam'])
print(f"The proportion of spam emails in training is: {proportion_spam:.4f}")

def prob_word_given_class(word, cls, word_frequency, class_frequency):
    """
    Calculate the conditional probability of a given word occurring in a specific class.
    """
    if cls == 'spam':
        amount_word_and_class = word_frequency.get(word, {'spam': 0})['spam']
    else:
        amount_word_and_class = word_frequency.get(word, {'ham': 0})['ham']
    
    p_word_given_class = amount_word_and_class / class_frequency[cls]
    return p_word_given_class

print(f"P(lottery | spam) = {prob_word_given_class('lottery', cls='spam', word_frequency=word_frequency, class_frequency=class_frequency)}")
print(f"P(lottery | ham) = {prob_word_given_class('lottery', cls='ham', word_frequency=word_frequency, class_frequency=class_frequency)}")
print(f"P(schedule | spam) = {prob_word_given_class('schedule', cls='spam', word_frequency=word_frequency, class_frequency=class_frequency)}")
print(f"P(schedule | ham) = {prob_word_given_class('schedule', cls='ham', word_frequency=word_frequency, class_frequency=class_frequency)}")

def prob_email_given_class(treated_email, cls, word_frequency, class_frequency):
    """
    Calculate the probability of an email being of a certain class based on treated email content.
    """
    prob = 1
    for word in treated_email:
        if word in word_frequency:
            prob *= prob_word_given_class(word, cls, word_frequency, class_frequency)
    return prob

def naive_bayes(treated_email, word_frequency, class_frequency, return_likelihood=False):    
    """
    Naive Bayes classifier for spam detection.
    """
    prob_email_given_spam = prob_email_given_class(treated_email, 'spam', word_frequency, class_frequency)
    prob_email_given_ham = prob_email_given_class(treated_email, 'ham', word_frequency, class_frequency)
    
    p_spam = class_frequency['spam'] / (class_frequency['ham'] + class_frequency['spam'])
    p_ham = class_frequency['ham'] / (class_frequency['ham'] + class_frequency['spam'])
    
    spam_likelihood = p_spam * prob_email_given_spam
    ham_likelihood = p_ham * prob_email_given_ham
    
    if return_likelihood:
        return (spam_likelihood, ham_likelihood)
    
    return 1 if spam_likelihood >= ham_likelihood else 0

# Test the classifier
example_email = "Click here to win a lottery ticket and claim your prize!"
treated_email = preprocess_text(example_email)[0]  # Get the first (and only) element of the list
print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nNaive Bayes predicts this email as: {naive_bayes(treated_email, word_frequency, class_frequency)}")

example_email = "Our meeting will happen in the main office. Please be there in time."
treated_email = preprocess_text(example_email)[0]
print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nNaive Bayes predicts this email as: {naive_bayes(treated_email, word_frequency, class_frequency)}")

def get_true_positives(Y_true, Y_pred):
    """
    Calculate the number of true positive instances in binary classification.
    """
    if len(Y_true) != len(Y_pred):
        return "Number of true labels and predicted labels must match!"
    
    true_positives = sum(1 for true, pred in zip(Y_true, Y_pred) if true == 1 and pred == 1)
    return true_positives

def get_true_negatives(Y_true, Y_pred):
    """
    Calculate the number of true negative instances in binary classification.
    """
    if len(Y_true) != len(Y_pred):
        return "Number of true labels and predicted labels must match!"
    
    true_negatives = sum(1 for true, pred in zip(Y_true, Y_pred) if true == 0 and pred == 0)
    return true_negatives

# Predictions for the test set
Y_pred = [naive_bayes(email, word_frequency, class_frequency) for email in X_test]

print(f"Y_test and Y_pred matches in length? Answer: {len(Y_pred) == len(Y_test)}")

true_positives = get_true_positives(Y_test, Y_pred)
true_negatives = get_true_negatives(Y_test, Y_pred)

print(f"The number of true positives is: {true_positives}\nThe number of true negatives is: {true_negatives}")

# Compute the accuracy
accuracy = (true_positives + true_negatives) / len(Y_test)
print(f"Accuracy: {accuracy:.4f}")


Number of emails: 5728
Proportion of spam emails: 0.2388
Proportion of ham emails: 0.7612
['re : energy derivatives conference - may 29 , toronto  good morning amy :  vince kaminski will need the following :  an lcd projector to hook up to a lap tap for his presentation  he will have dinner with the conference organizers and speakers on the 29 th .  he will need 2 nights ( the 28 th and the 29 th ) hotel reservations .  he will send you an abstract shortly .  thanks and have a great day !  shirley crenshaw  713 - 853 - 5290  amy aldous on 03 / 31 / 2000 10 : 50 : 11 am  to : shirley . crenshaw @ enron . com  cc :  subject : re : energy derivatives conference - may 29 , toronto  ms . crenshaw ,  thank you for sending the bio so quickly . it \' s exactly what i was looking  for .  we are planning to compile the conference speakers \' papers for distribution  to the participants . while i will not need dr . kaminski \' s contribution for  several weeks , an abstract of his presentation as